In [2]:
pip install cufflinks

  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68727 sha256=c94848570291e634069c8116770e6e2017d0c092e9374129dc11b3f1f534376b
  Stored in directory: c:\users\이신행\appdata\local\pip\cache\wheels\6b\76\62\6da97734911ffcbdd559fd1a3f28526321f0ae699182a23866
Successfully built cufflinks
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import cufflinks as cf #  pandas 형태의 데이터를 plotly를 이용해 쉽게 시각화 할 수 있도록 도와주는 라이브러리
from scipy import stats # 여러가지 확률 분포 분석을 위한 scipy 패키지
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()

## 데이터 불러오기

In [16]:
PopularCreativeTypes = pd.read_csv("C:/Users/이신행/OneDrive/바탕 화면/archive/PopularCreativeTypes.csv");
HighestGrossers = pd.read_csv("C:/Users/이신행/OneDrive/바탕 화면/archive/HighestGrossers.csv");
Data_Annual_Ticket = pd.read_csv("C:/Users/이신행/OneDrive/바탕 화면/archive/AnnualTicketSales.csv",thousands = ',');

## 데이터  처리 및 정보 표시

In [17]:
# 지정 데이터의 ,을 빈칸으로 대체
Data_Annual_Ticket["TICKETS SOLD"] = Data_Annual_Ticket["TICKETS SOLD"].replace(',','')

Data_Annual_Ticket["TOTAL BOX OFFICE"] = Data_Annual_Ticket["TOTAL BOX OFFICE"].str.replace(',','')
Data_Annual_Ticket["TOTAL BOX OFFICE"] = Data_Annual_Ticket["TOTAL BOX OFFICE"].str.replace('$','')

Data_Annual_Ticket["TOTAL INFLATION ADJUSTED BOX OFFICE"] = Data_Annual_Ticket["TOTAL INFLATION ADJUSTED BOX OFFICE"].str.replace(',','')
Data_Annual_Ticket["TOTAL INFLATION ADJUSTED BOX OFFICE"] = Data_Annual_Ticket["TOTAL INFLATION ADJUSTED BOX OFFICE"].str.replace('$','')

Data_Annual_Ticket["AVERAGE TICKET PRICE"] = Data_Annual_Ticket["AVERAGE TICKET PRICE"].str.replace('$','')
# unnamed 5 칼럼 제거
Data_Annual_Ticket = Data_Annual_Ticket.drop(labels = "Unnamed: 5",axis = 1)

<ipython-input-17-0919e9d7b091>:5: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-17-0919e9d7b091>:8: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-17-0919e9d7b091>:10: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [18]:
Data_Annual_Ticket.head(5)

,YEAR,TICKETS SOLD,TOTAL BOX OFFICE,TOTAL INFLATION ADJUSTED BOX OFFICE,AVERAGE TICKET PRICE
0,2021,423774881,3881777912,3881777912,9.16
1,2020,223638958,2048534616,2048534616,9.16
2,2019,1228541629,11253443955,11253444050,9.16
3,2018,1311536128,11948096650,12013670952,9.11
4,2017,1225639761,10993991460,11226860216,8.97


## 데이터 타입 변경 (object -> float)

In [19]:
Data_Annual_Ticket['TICKETS SOLD'] = Data_Annual_Ticket['TICKETS SOLD'].astype(float)
Data_Annual_Ticket['TOTAL BOX OFFICE'] = Data_Annual_Ticket['TOTAL BOX OFFICE'].astype(float)

## 막대 차트를 사용하여 매년 전체 박스 오피스 표시

In [20]:
px.bar(Data_Annual_Ticket,x='YEAR',y='TOTAL BOX OFFICE',title='Total Box Office vs . Year')

## 지난 2년이 정규 연도인 경우 총 박스 오피스 계산(선형 회귀 분석 사용)

In [22]:
x = list(range(0,(2020-1995)))
y = list(Data_Annual_Ticket['TOTAL BOX OFFICE'] )
y.reverse() # 리스트를 역순으로
y.pop() # 맨 뒤에 있는 값 나타내고 제거
y.pop()
slope, intercept, r, p, std_err = stats.linregress(x, y)
x1 = list(range(0,(2022-1995)))
y1= [slope * x + intercept for x in x1 ]
y1.reverse()
Data_Annual_Ticket['TOTAL BOX OFFICE WITHOUT COVID'] = y1
Data_Annual_Ticket["Diff"] = Data_Annual_Ticket['TOTAL BOX OFFICE WITHOUT COVID']-Data_Annual_Ticket['TOTAL BOX OFFICE']

## 코로나 19로 인한 흥행 비교

In [24]:
px.line(Data_Annual_Ticket,x='YEAR',y=['TOTAL BOX OFFICE','TOTAL BOX OFFICE WITHOUT COVID'],
       labels={'YEAR':"Years",'value':'Total Sale'},title='TOTAL BOX OFFICE vs TOTAL BOX OFFICE WITHOUT COVID')

## 지난 2년간 covid-19가 얼마나 영향을 미치는지 확인

In [25]:
# 코로나 19로 인한 매출 손실 그래프
px.bar(Data_Annual_Ticket
       ,x = 'YEAR'
       ,y = "Diff"
       ,labels = {'YEAR' :"Year", "Diff": "Financial Loss"}
       ,title = 'Financial Loss (just last two years are important)'
       ,barmode='group')

## covid-19가 지난 2년간 전체 박스오피스에 미치는 영향은?

In [26]:
Data_Annual_Ticket['Percentage of Financial Loss'] = (Data_Annual_Ticket['TOTAL BOX OFFICE WITHOUT COVID']-
                                                     Data_Annual_Ticket['TOTAL BOX OFFICE'])/Data_Annual_Ticket['TOTAL BOX OFFICE WITHOUT COVID'] * 100

px.bar(Data_Annual_Ticket,x='YEAR', y='Percentage of Financial Loss',
      labels={'YEAR' :"Year", "Percentage of Financial Loss": "Percentage of Financial Loss %"},
      title='Financial Loss % (just last two years are important)')

## 이제 가장 많은 수익을 창출하는 기업 시각화

## 데이터 처리 및 정보 표시

In [29]:
HighestGrossers["TOTAL IN 2019 DOLLARS"] = HighestGrossers["TOTAL IN 2019 DOLLARS"].str.replace(',','')
HighestGrossers["TOTAL IN 2019 DOLLARS"] = HighestGrossers["TOTAL IN 2019 DOLLARS"].str.replace('$','')

HighestGrossers["TICKETS SOLD"] = HighestGrossers["TICKETS SOLD"].str.replace(',','')

HighestGrossers['TOTAL IN 2019 DOLLARS'] = HighestGrossers['TOTAL IN 2019 DOLLARS'].astype(float)
HighestGrossers['TICKETS SOLD'] = HighestGrossers['TICKETS SOLD'].astype(float)

<ipython-input-29-8afdb6605a50>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [31]:
HighestGrossers.head(5)

,YEAR,MOVIE,GENRE,MPAA RATING,DISTRIBUTOR,TOTAL FOR YEAR,TOTAL IN 2019 DOLLARS,TICKETS SOLD
0,1995,Batman Forever,Drama,PG-13,Warner Bros.,"$184,031,112",387522978.0,42306002.0
1,1996,Independence Day,Adventure,PG-13,20th Century Fox,"$306,169,255",634504608.0,69269062.0
2,1997,Men in Black,Adventure,PG-13,Sony Pictures,"$250,650,052",500207943.0,54607854.0
3,1998,Titanic,Adventure,PG-13,Paramount Pictures,"$443,319,081",865842808.0,94524324.0
4,1999,Star Wars Ep. I: The Phantom Menace,Adventure,PG,20th Century Fox,"$430,443,350",776153749.0,84732942.0


## 인플레이션 때문에 2019 달러 컬럼에 토탈을 사용
## 서로 다른 것들의 비율을 설명하기 위해 파이 차트를 사용

In [32]:
px.pie(HighestGrossers,values='TOTAL IN 2019 DOLLARS',names='DISTRIBUTOR',
      title='Percentage of Each Distributors in Total Ticket Sale',
      color_discrete_sequence=px.colors.sequential.RdBu,
      height=600)

In [33]:
px.pie(HighestGrossers
       ,values = 'TOTAL IN 2019 DOLLARS' 
       ,names = 'MPAA RATING'
       ,title = 'Percentage of Each MPAA Rating in Total Ticket Sale'
       ,color_discrete_sequence = px.colors.sequential.RdBu
       ,height = 600
       )

## 막대 차트를 사용하여 각 제작사과 각 장르에 대한 총 티켓 판매의 합계를 설명

In [36]:
df_g=HighestGrossers.groupby(by=['DISTRIBUTOR','GENRE'])['TICKETS SOLD'].sum().unstack().iplot(kind='bar')
# unstack : 열방향으로 구조화

## 막대 차트를 사용하여 각 제작사 및 각 장르에 대한 총 티켓 판매 수를 명시합니다.

In [37]:
df_g = HighestGrossers.groupby(by =['DISTRIBUTOR','GENRE'])['TICKETS SOLD'].count().unstack().iplot(kind = 'bar')

## MPAA 등급에 동일한 작업을 수행.

In [38]:
df_g = HighestGrossers.groupby(by =['DISTRIBUTOR','MPAA RATING'])['TICKETS SOLD'].sum().unstack().iplot(kind = 'bar')

In [39]:
df_g = HighestGrossers.groupby(by =['DISTRIBUTOR','MPAA RATING'])['TICKETS SOLD'].count().unstack().iplot(kind = 'bar')

## 인기있는 CREATIVE TYPES 타입을 시각화

In [41]:
PopularCreativeTypes.head(5)

,RANK,CREATIVE TYPES,MOVIES,TOTAL GROSS,AVERAGE GROSS,MARKET SHARE
0,1.0,Contemporary Fiction,"7,442","$96,203,727,036","$12,927,133",40.46%
1,2.0,Kids Fiction,564,"$32,035,539,746","$56,800,602",13.47%
2,3.0,Science Fiction,724,"$29,922,660,857","$41,329,642",12.59%
3,4.0,Fantasy,759,"$21,724,062,575","$28,621,953",9.14%
4,5.0,Super Hero,129,"$20,273,157,911","$157,156,263",8.53%


In [42]:
# ,와 $ 제거하고 표시
PopularCreativeTypes["TOTAL GROSS"] = PopularCreativeTypes["TOTAL GROSS"].str.replace(',','')
PopularCreativeTypes["TOTAL GROSS"] = PopularCreativeTypes["TOTAL GROSS"].str.replace('$','')

PopularCreativeTypes["AVERAGE GROSS"] = PopularCreativeTypes["AVERAGE GROSS"].str.replace(',','')
PopularCreativeTypes["AVERAGE GROSS"] = PopularCreativeTypes["AVERAGE GROSS"].str.replace('$','')

PopularCreativeTypes["MARKET SHARE"] = PopularCreativeTypes["MARKET SHARE"].str.replace('%','')

PopularCreativeTypes["MOVIES"] = PopularCreativeTypes["MOVIES"].str.replace(',','')

<ipython-input-42-7af4d0a9fd3f>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-42-7af4d0a9fd3f>:5: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [43]:
# 불필요 칼럼 제거
PopularCreativeTypes = PopularCreativeTypes.drop(index = 9,axis = 0)

In [44]:
# 데이터 타입 변경
PopularCreativeTypes["MOVIES"] = PopularCreativeTypes["MOVIES"].astype(float)
PopularCreativeTypes["TOTAL GROSS"] = PopularCreativeTypes["TOTAL GROSS"].astype(float)
PopularCreativeTypes["AVERAGE GROSS"] = PopularCreativeTypes["AVERAGE GROSS"].astype(float)
PopularCreativeTypes["MARKET SHARE"] = PopularCreativeTypes["MARKET SHARE"].astype(float)

## 어떤 creative types가 가장 성장 비중이 높은지 확인

In [45]:
px.pie(PopularCreativeTypes,values='TOTAL GROSS',names='CREATIVE TYPES',
      title='Percentage of Creative Types in Total Gross',
      color_discrete_sequence=px.colors.sequential.RdBu,height=600)

In [46]:
px.bar(PopularCreativeTypes
      ,x = "TOTAL GROSS"
      ,y ="CREATIVE TYPES"
      ,title = "Total Gross of Different type")

## 어떤 CREATIVE TYPES가 평균 성장이 높은지 확인

In [47]:
px.pie(PopularCreativeTypes,values='AVERAGE GROSS',names='CREATIVE TYPES',title='Percentage of Creative Types in Average Gross',
      color_discrete_sequence=px.colors.sequential.RdBu,height=600)

In [48]:
px.bar(PopularCreativeTypes
      ,x = "AVERAGE GROSS"
      ,y = "CREATIVE TYPES"
      ,title = "Average Gross in Different type")

## 매 해 어떤 영화의 비중이 높은지 확인

In [51]:
px.pie(PopularCreativeTypes
       ,values = 'MOVIES' 
       ,names = 'CREATIVE TYPES'
       ,title = 'Percentage of Number of Movies in Each Types'
       ,color_discrete_sequence = px.colors.sequential.RdBu
       ,height = 600
       )

In [50]:
px.bar(PopularCreativeTypes
      ,x = "MOVIES"
      ,y ="CREATIVE TYPES"
      ,title = "Number of Muvies in Different type")